In [1]:
import anndata
import pixelator
import torch
import scvi
import scipy
# from scvi import autotune

import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection

# import ray
# from ray import tune


from PixelGen.pxl_utils import *
from PixelGen.scvi_utils import plot_losses
from PixelGen.tests import test_convert_edgelist_to_protein_pair_colocalization

from pixelator.plot import molecule_rank_plot, cell_count_plot, scatter_umi_per_upia_vs_tau
from pixelator.statistics import clr_transformation
from pixelator.analysis.normalization import dsb_normalize

from scipy.sparse.csgraph import dijkstra
import hotspot

import tempfile

from scvi import REGISTRY_KEYS
from scvi.module.base import (
    BaseModuleClass,
    LossOutput,
    PyroBaseModuleClass,
    auto_move_data,
)
from torch.distributions import NegativeBinomial, Normal, Poisson
from torch.distributions import kl_divergence as kl

from PixelGen.modules import MultiModalSCVI

print(torch.cuda.is_available())

%load_ext autoreload
%autoreload 2

scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)
sc.set_figure_params(figsize=(6, 6), frameon=False)
sns.set_theme()
torch.set_float32_matmul_precision("high")
save_dir = tempfile.TemporaryDirectory()

%config InlineBackend.print_figure_kwargs={"facecolor": "w"}
%config InlineBackend.figure_format="retina"

True


Seed set to 0


Last run with scvi-tools version: 1.2.2.post2


In [16]:
DATA_DIR = Path('./PixelGen/datasets')


FILENAMES = [
    "Sample05_V2_PBMC_r1.layout.dataset.pxl",
    "Sample06_V2_PBMC_r2.layout.dataset.pxl",
    "Sample07_V2_PHA_PBMC_r1.layout.dataset.pxl",
    "Sample08_V2_PHA_PBMC_r2.layout.dataset.pxl",
]

SAMPLE_NAMES = [
    "resting_r1", 
    "resting_r2",
    "stimulated_r1", 
    "stimulated_r2",
]

# COMBINED_FILENAME = "combined_resting_data.pxl"

COMBINED_FILENAME = "combined_resting_PHA_data.pxl"
COMBINED_PROCESSED_NAME = "combined_resting_PHA_data_PROCESSED.pxl"

combined_path = DATA_DIR / COMBINED_FILENAME

# pg_data = pixelator.read(DATA_DIR / COMBINED_PROCESSED_NAME)
# adata = pg_data.adata

if os.path.exists(combined_path):
    pg_data = pixelator.read(combined_path)
else:
    
    baseurl = "https://pixelgen-technologies-datasets.s3.eu-north-1.amazonaws.com/mpx-datasets/pixelator/0.18.x/uropod-t-cells-v1.0-immunology-I"
    for filename in FILENAMES:
        !curl -L -O -C - --create-dirs --output-dir {DATA_DIR} {baseurl}/{filename}

    datasets = [pixelator.read(DATA_DIR / filename) for filename in FILENAMES]
    pg_data = pixelator.simple_aggregate(
        SAMPLE_NAMES, datasets
    )
    pg_data.save(combined_path, force_overwrite=True)

In [17]:
adata = anndata.read_h5ad('./PixelGen/datasets/combined_data_coloc_with_z.h5ad')
adata

AnnData object with n_obs × n_vars = 100 × 75
    obs: 'pixels', 'a_pixels', 'b_pixels', 'antibodies', 'molecules', 'reads', 'mean_reads_per_molecule', 'median_reads_per_molecule', 'mean_b_pixels_per_a_pixel', 'median_b_pixels_per_a_pixel', 'mean_a_pixels_per_b_pixel', 'median_a_pixels_per_b_pixel', 'a_pixel_b_pixel_ratio', 'mean_molecules_per_a_pixel', 'median_molecules_per_a_pixel', 'leiden', 'tau_type', 'tau', 'sample', 'n_antibodies_by_counts', 'log1p_n_antibodies_by_counts', 'total_counts', 'log1p_total_counts', 'sample_class', 'cell_type'
    var: 'antibody_count', 'components', 'antibody_pct', 'nuclear', 'control', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'cell_type_colors', 'coloc_info', 'counts_per_sample', 'dendrogram_leiden', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'sample_class_colors', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap', 'coloc', 'counts_and_c

In [10]:
adata = pg_data.adata
adata.obs['condition'] = adata.obs['sample'].map({s: 'stim' if 'stimulated' in s else 'resting' for s in adata.obs['sample'].unique()})
sc.pp.pca(adata, layer='dsb')
sc.pp.neighbors(adata,)
sc.tl.umap(adata)
sc.pl.umap(adata, color=['condition', 'cell_type'])

KeyError: 'dsb'